# Modelo Autorregresivo de medias móviles (ARMA)

Repositorio de referencia
https://github.com/ecabestadistica/curso-series-temporales/

El modelo ARMA se usa como modelo de comparación en el artículo: Hourly global solar irradiation forecasting for New Zealand

Se considera el valor de la variable y del residuo del periodo anterior.

$x_{t}=c+ϕ_{1} x_{t-1}+θ _{1} ϵ _{t-1}+ϵ_{t}$

Donde $ϵ _{t-1}$ es el residuo del período anterior.

$|θ_{1}|<1$

$| ϕ_{1} | < 1$

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.arima_model import ARMA
from scipy.stats.distributions import chi2 
from math import sqrt
import seaborn as sns
sns.set()

In [ ]:
#df2015 = pd.read_csv('./drive/MyDrive/DatosInvestigacion/NoNull2015.csv', index_col=0, parse_dates=True)
df2015 = pd.read_csv('./drive/MyDrive/DatosInvestigacion/DayData2015.csv', index_col=0, parse_dates=True)

In [ ]:
df = pd.DataFrame()
df['RSG'] = df2015['SRGLOAVG1H']

In [ ]:
df

,RSG
Local_Time,
2015-01-01 07:00:00,15.0
2015-01-01 08:00:00,107.0
2015-01-01 09:00:00,394.0
2015-01-01 10:00:00,728.0
2015-01-01 11:00:00,428.0
...,...
2015-12-31 14:00:00,472.0
2015-12-31 15:00:00,396.0
2015-12-31 16:00:00,419.0


## LLR Test

In [ ]:
def LLR_test(mod_1, mod_2, DF = 1):
    L1 = mod_1.llf
    L2 = mod_2.llf
    # estadístico
    LR = (2*(L2-L1))    
    p = chi2.sf(LR, DF).round(3)
    return p

In [ ]:
# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

## ARMA(1,1)

In [ ]:
model_arma11 = ARMA(df['RSG'], order=(1,1))
results_arma11 = model_arma11.fit()
results_arma11.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              ARMA Model Results                              
==============================================================================
Dep. Variable:                    RSG   No. Observations:                 4380
Model:                     ARMA(1, 1)   Log Likelihood              -28582.178
Method:                       css-mle   S.D. of innovations            165.102
Date:                Mon, 13 Dec 2021   AIC                          57172.356
Time:                        18:55:59   BIC                          57197.896
Sample:                             0   HQIC                         57181.368
                                                                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        344.0496      8.765     39.252      0.000     326.870     361.229
ar.L1.RSG      0.6324      0.014     44.217      0.000       0.604       0.660
ma.L1.RSG      0.2922      0.016     18.004      0.000       0.260       0.324
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.5813           +0.0000j            1.5813            0.0000
MA.1           -3.4226           +0.0000j            3.4226            0.5000
-----------------------------------------------------------------------------
"""

Los p-valores son 0, así que los coeficientes son significativas.

In [ ]:
model_ar = ARMA(df['RSG'], order=(1,0))
results_ar = model_ar.fit()
model_ma = ARMA(df['RSG'], order=(0,1))
results_ma = model_ma.fit()


El modelo más complejo obtiene resultados significativamente mejores.

In [ ]:
print("\nARMA vs AR ", LLR_test(results_ar, results_arma))
print("\nARMA vs MA ", LLR_test(results_ma, results_arma))


ARMA vs AR  0.0

ARMA vs MA  0.0


## ARMA(p,q)

In [ ]:
model_arma23 = ARMA(df['RSG'], order=(2,3)) #articulo

In [ ]:
results_arma23 = model_arma23.fit()
results_arma23.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              ARMA Model Results                              
==============================================================================
Dep. Variable:                    RSG   No. Observations:                 4380
Model:                     ARMA(2, 3)   Log Likelihood              -27988.265
Method:                       css-mle   S.D. of innovations            144.119
Date:                Mon, 13 Dec 2021   AIC                          55990.529
Time:                        18:56:08   BIC                          56035.223
Sample:                             0   HQIC                         56006.299
                                                                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        344.2764      3.608     95.429      0.000     337.206     351.347
ar.L1.RSG      1.7256      0.002    841.070      0.000       1.722       1.730
ar.L2.RSG     -0.9931      0.002   -485.100      0.000      -0.997      -0.989
ma.L1.RSG     -1.1112      0.013    -85.009      0.000      -1.137      -1.086
ma.L2.RSG      0.0836      0.019      4.414      0.000       0.046       0.121
ma.L3.RSG      0.4707      0.012     37.758      0.000       0.446       0.495
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            0.8688           -0.5021j            1.0035           -0.0834
AR.2            0.8688           +0.5021j            1.0035            0.0834
MA.1            0.8855           -0.5533j            1.0441           -0.0889
MA.2            0.8855           +0.5533j            1.0441            0.0889
MA.3           -1.9485           -0.0000j            1.9485           -0.5000
-----------------------------------------------------------------------------
"""

El resultado es significativamente diferente

In [ ]:
print("\nARMA(2,3) vs ARMA(1,1) ", LLR_test(results_arma11, results_arma23,3))


ARMA(2,3) vs ARMA(1,1)  0.0


Correlación:
* Max temperature (Tmax) 0.462
* Min temperature (Tmin) 0.404
* Relative humidity (RH) 0.505
* Rain amount (Ra) 0.055
* Solar zenith angle (SZA) 0.066
* Azimuth (Az) 0.183
* Pressure (P) 0.020
* Wind speed (Ws) 0.002
* Wind direction (Wd) 0.005

Misma arquitectura - dos modelos: Difusa, Global

* Curva mensual
* Diagramas de cajas a nivel mensual
* Diagrama de cajas a nivel horario

Mapa de correlaciones: seaborn clustermap